## Set up

In [ ]:
import sys
sys.path.append('./scripts/')
import os
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.axes as axes
import seaborn as sns
import math
import copy
import numpy as np
sns.set_style("darkgrid")
from PIL import Image
import random # random seed to reproduce MDS and t-SNE plots

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import cluster # k-Means clustering
from sklearn.cluster import KMeans
from sklearn import manifold # MDS and t-SNE
from sklearn.metrics import silhouette_score # silhouette width for clustering
from sklearn import preprocessing # scaling attributes
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics.pairwise import pairwise_distances
import hdbscan
import umap

import torch
import torchvision

# from captum.concept import TCAV
# from captum.concept import Concept
# from captum.concept._utils.data_iterator import dataset_to_dataloader, CustomIterableDataset
# from captum.concept._utils.common import concepts_to_str

from lucent.optvis import render, param, transform, objectives

import imp
import my_datasets
import utilities 
imp.reload(my_datasets) 
imp.reload(utilities) 

plt.rcParams["figure.figsize"] = (3,3)
random.seed(2023)

In [ ]:
dataset='ilsvrc12fine'
paths, count, y, idx_to_labels = my_datasets.get_dataset(dataset)

print(count, len(paths))

In [ ]:
# For ilsvrc12fine dataset, paths are mapped differently
if dataset=='ilsvrc12fine':
    idxs=np.arange(0, 1281167, 10) 
    classes=np.unique(y[idxs])
    ppaths=[paths[i] for i in idxs]
    paths=ppaths

In [ ]:
layer='Mixed_7b.cat_2'
SAVEFOLD0=f'../outputs/{dataset}'
SAVEFOLD=f"{SAVEFOLD0}/{layer}/"

In [ ]:
#gradients_wrt_conv_layer=np.load(f"{SAVEFOLD}/gradients_wrt_conv_layer.npy")
predictions=np.load(f"{SAVEFOLD}/predictions.npy", mmap_mode = 'r')
conv_maps=np.load(f"{SAVEFOLD}/conv_maps.npy", mmap_mode = 'r')

# pvh=np.load(f"{SAVEFOLD}/eigenvectors.npy",allow_pickle=True, mmap_mode = 'r')

In [ ]:
conv_maps_avg = conv_maps.mean(3).mean(2)

In [ ]:
# pu, ps, pvh = np.linalg.svd(conv_maps_avg)

# np.save(f"{SAVEFOLD}/pu.npy", pu)
# np.save(f"{SAVEFOLD}/ps.npy", ps)
# np.save(f"{SAVEFOLD}/eigenvectors.npy", pvh)

In [ ]:
pvh = np.load(f'{SAVEFOLD}/eigenvectors.npy')
pu = np.load(f'{SAVEFOLD}/pu.npy')
ps = np.load(f'{SAVEFOLD}/ps.npy')

Are we standardising or normalising the activations?

In [ ]:
transforms = None # None / "standardise" / "normalise"

Global average pooling of activations

In [ ]:
scale = StandardScaler()
normalise = MinMaxScaler()

standardised_data = scale.fit_transform(conv_maps_avg) 
normalised_data = normalise.fit_transform(conv_maps_avg) # .shape (10000, 2048)

In [ ]:
if transforms == "standardise":
    activations = standardised_data
    print("Standardise")
elif transforms == "normalise": 
    activations = normalised_data
    print("Normalise")
else: 
    activations = conv_maps_avg
    print("Raw activations")

In [ ]:
# conv_maps_avg

In [ ]:
# activations

## Utilities

## Random analysis

Evec maximally projecting images

In [ ]:
# num_dirs = 10
# top=50
# evecs_dot = np.empty([len(conv_maps_avg),num_dirs])
# evecs_sim = np.empty([len(conv_maps_avg),num_dirs])
# for i in range(len(conv_maps_avg)):
#     for direction in range(num_dirs):
#         evecs_dot[i,direction] = np.dot(conv_maps_avg[i], pvh[direction])
#         evecs_sim[i,direction] = evecs_dot[i,direction]/(np.linalg.norm(conv_maps_avg[i])*np.linalg.norm(conv_maps_avg[direction]))

# top_evec_projs = []
# for direction in range(len(evecs_dot[0,])):
#     top_evec_projs.append(evecs_dot[:,direction].argsort()[-top:][::-1])
    
# for direction in range(num_dirs):
#     fig, ax = plt.subplots(math.ceil(top//5), 5, figsize = (10,20))
#     ax = ax.flatten()
#     for idx, im_id in enumerate(top_evec_projs[direction]):# enumerate(concepts_dot[:,concept].argsort()[-top:][::-1]):
#         im = Image.open(paths[im_id])
#         ax[idx].imshow(im)
#         ax[idx].set_title(f"{im_id}", size = 8)
#         ax[idx].axis('off')

Analyse sample of random images

In [ ]:
top=100
rand_ims = []
for i in range(top):
    rand_ims.append(random.randint(0, len(paths)))
rand_ims = np.array(rand_ims)
rand_ims
rand_activations = utilities.get_activations(activations_avg = activations, ims=rand_ims)

clusterer_rand = AgglomerativeClustering(n_clusters=None, distance_threshold=0, metric=metric,linkage=linkage )
clusterer_rand.fit_predict(rand_activations)

linkage_res = utilities.plot_dendrogram(clusterer_rand, truncate_mode="level") # , p=100

## Settings

Neurons:

1 underwater, scuba diving

5 peacock/police van

13 black background, clusters for performing

16 oval / lobster

18 snake, coral

25 writing sticking out and print

27 fluffy cream dog, book shelves

22 toilet roll, brown dog face

35 sport/green apple

57 black dog, diagonal rod


In [ ]:
# image collection params
direction = 22
top = 100

# clustering params
linkage='ward'
metric='euclidean'
distance_threshold = 15

kmeans_outlier_threshold = 15
min_ims_cluster = 5

In [ ]:
# # print(clusterer_rand.distances_)
# _ = utilities.plot_euclidan_distances(rand_activations, label = 'random images')

# Step 1: Calculate top images and extract activations for selected neuron

In [ ]:
#sanity check clustering random images
# top_ims = []
# for i in range(top):
#     top_ims.append(random.randint(0, len(paths)))
# top_ims = np.array(top_ims)
# top_ims

#### If looking at neuron

In [ ]:
top_ims = utilities.get_activations(activations_avg = activations, direction = direction).argsort()[-top:][::-1] 
top_activations = utilities.get_activations(activations_avg = activations, ims=top_ims)

#### If looking at evec direction

In [ ]:
#in this case direction is evec direction
# evec_dot = np.empty([len(conv_maps_avg)])
# evec_sim = np.empty([len(conv_maps_avg)])
# for i in range(len(conv_maps_avg)):
#     evec_dot[i] = np.dot(conv_maps_avg[i], pvh[direction])
#     evec_sim[i] = evec_dot[i]/(np.linalg.norm(conv_maps_avg[i])*np.linalg.norm(conv_maps_avg[direction]))

# top_ims = evec_dot.argsort()[-top:][::-1]
# top_activations = utilities.get_activations(activations_avg = activations, ims=top_ims)
    
# fig, ax = plt.subplots(math.ceil(top//5), 5, figsize = (10,20))
# ax = ax.flatten()
# for idx, im_id in enumerate(top_ims):# enumerate(concepts_dot[:,concept].argsort()[-top:][::-1]):
#     im = Image.open(paths[im_id])
#     ax[idx].imshow(im)
#     ax[idx].set_title(f"{im_id}", size = 8)
#     ax[idx].axis('off')

## Step 2: Find number of clusters using agglomerative clustering

First, just look dendrogram

In [ ]:
clusterer_0 = AgglomerativeClustering(n_clusters=None, distance_threshold=0, metric=metric,linkage=linkage )
clusterer_0.fit_predict(top_activations)

In [ ]:
linkage_res = utilities.plot_dendrogram(clusterer_0, truncate_mode="level") # , p=100 # 

In [ ]:
# clusterer_0.distances_

Cluster with set distance threshold

In [ ]:
clusterer = AgglomerativeClustering(n_clusters=None, distance_threshold=distance_threshold, metric=metric,linkage=linkage)
clusterer.fit_predict(top_activations)

In [ ]:

#old - taking longest clean distance in dendrogram
# max_diff = [clusterer_0.distances_[i] - clusterer_0.distances_[i-1] if i else clusterer_0.distances_[i] for i in range(len(clusterer_0.distances_))]  
# clusterer = AgglomerativeClustering(n_clusters=None, distance_threshold=clusterer_0.distances_[np.argmax(max_diff)], metric=metric,linkage=linkage )

#RW
# clusterer = AgglomerativeClustering(n_clusters=None, distance_threshold=12, metric=metric,linkage=linkage )
# clusterer.fit_predict(top_activations)

Visualise how hierarchial clustering clusters number of clusters selected

In [ ]:
clu_labs = clusterer.labels_
# print(clu_labs)
clu_lab_order = sorted(range(len(clu_labs)), key=lambda k: clu_labs[k])

fig, ax = plt.subplots(math.ceil(len(top_ims)//5), 5, figsize = (10,20))
ax = ax.flatten()
for idx, im_id in enumerate(top_ims[clu_lab_order]):
    im = Image.open(paths[im_id])
    ax[idx].imshow(im)
    ax[idx].set_title(f"{im_id}: cluster {clu_labs[clu_lab_order][idx]}", size = 8)
    ax[idx].axis('off')

## Step 3: Run kmeans with number of clusters from step 2, remove outliers

Cluster with number of clusters selected with kmeans since need centroids to remove outliers

In [ ]:
kmeans = KMeans(n_clusters=clusterer.n_clusters_, random_state=0, n_init=5, max_iter=1000).fit(top_activations)
# kmeans # "auto"

plot kmeans clusters before removing outliers

In [ ]:
clu_labs = kmeans.labels_
# print(clu_labs)
clu_lab_order = sorted(range(len(clu_labs)), key=lambda k: clu_labs[k])

fig, ax = plt.subplots(math.ceil(len(top_ims)//5), 5, figsize = (10,20))
ax = ax.flatten()
for idx, im_id in enumerate(top_ims[clu_lab_order]):
    im = Image.open(paths[im_id])
    ax[idx].imshow(im)
    ax[idx].set_title(f"{im_id}: cluster {clu_labs[clu_lab_order][idx]}", size = 8)
    ax[idx].axis('off')

Visualise activations with UMAP

In [ ]:
XY_UMAP = umap.UMAP(n_components=2).fit_transform(top_activations)
utilities.clustering_scatterplot(points=XY_UMAP, 
                       labels=clu_labs,
                       centers=None, 
                       title='UMAP')

In [ ]:
cosine_sim = utilities.plot_cosine_similarities(top_ims, min_sim=0, max_sim=1, maps = activations, label = 'All')
cosine_sim = utilities.plot_cosine_similarities(top_ims[clu_lab_order], min_sim=0, max_sim=1, maps = activations, label = 'Ordered')
# for cluster in np.unique(clu_labs): 
#     cosine_sim = utilities.plot_cosine_similarities(top_ims[clu_labs==cluster], min_sim=0, max_sim=1, maps = activations, label = f'Cluster {round(cluster)}')

In [ ]:
distance_matrix = utilities.plot_euclidan_distances(top_activations, label = 'all')
distance_matrix = utilities.plot_euclidan_distances(top_activations[clu_lab_order], label = 'all')
# for cluster in np.unique(clu_labs):
#     distance_matrix = utilities.plot_euclidan_distances(top_activations[clu_labs==cluster], label = f'cluster {round(cluster)}')

Need to remove outliers to purify clusters

Get squared distance to kmeans centroid of appropriate cluster - transform()

https://stackoverflow.com/questions/54240144/distance-between-nodes-and-the-centroid-in-a-kmeans-cluster

In [ ]:
centroid_dist = kmeans.transform(top_activations)**2
nearest_centroid_dist = np.zeros(len(clu_labs))
nearest_centroid_dist = [centroid_dist[i,clu_labs[i]] for i in range(len(clu_labs))]
# nearest_centroid_dist
# kmeans.cluster_centers_

Calculate which images are outliers by removing observations far away from centroid

Rename outliers to now belong to cluster label '100'. They will now be ordered at the end. 

In [ ]:
clu_labs_rm_outliers = np.empty(len(clu_labs))
clu_labs_rm_outliers[:] = 100
# clu_labs_rm_outliers
#clu_labs_rm_outliers = [centroid_dist[i,clu_labs[i]] for i in range(len(clu_labs))]
for i in range(len(clu_labs)):
    if nearest_centroid_dist[i] < kmeans_outlier_threshold:
        clu_labs_rm_outliers[i] = clu_labs[i]

visualise images remaining in clusters after removing outliers

In [ ]:
# clu_labs_rm_outliers_order = sorted(range(len(clu_labs_rm_outliers)), key=lambda k: clu_labs_rm_outliers[k])

# fig, ax = plt.subplots(math.ceil(len(top_ims)//5), 5, figsize = (10,20))
# ax = ax.flatten()
# for idx, im_id in enumerate(top_ims[clu_labs_rm_outliers_order]):
#     im = Image.open(paths[im_id])
#     ax[idx].imshow(im)
#     ax[idx].set_title(f"{im_id}: cluster {clu_labs_rm_outliers[clu_labs_rm_outliers_order][idx]}", size = 8)
#     ax[idx].axis('off')

Remove clusters with less than treshold number of items

In [ ]:
for cluster in range(clusterer.n_clusters_):
    count = sum(clu_labs_rm_outliers == cluster)
    # print(count)
    if count < min_ims_cluster: 
        # print(cluster)
        #clu_labs_rm_outliers[i] = clu_labs[i]
        clu_labs_rm_outliers[[clu_labs_rm_outliers[i] == cluster for i in range(len(clu_labs_rm_outliers))]] = 100

visualise images remaining in clusters after removing small clusters

In [ ]:
clu_labs_rm_outliers_order = sorted(range(len(clu_labs_rm_outliers)), key=lambda k: clu_labs_rm_outliers[k])

fig, ax = plt.subplots(math.ceil(len(top_ims)//5), 5, figsize = (10,20))
ax = ax.flatten()
for idx, im_id in enumerate(top_ims[clu_labs_rm_outliers_order]):
    im = Image.open(paths[im_id])
    ax[idx].imshow(im)
    ax[idx].set_title(f"{im_id}: cluster {clu_labs_rm_outliers[clu_labs_rm_outliers_order][idx]}", size = 8)
    ax[idx].axis('off')

Plot cosine similarities for all top images and clusters

In [ ]:
cosine_sim = utilities.plot_cosine_similarities(top_ims, min_sim=0, max_sim=1, maps = activations, label = 'all')
# cosine_sim = utilities.plot_cosine_similarities(top_ims[clu_labs_rm_outliers_order], min_sim=0, max_sim=1, maps = activations, label = 'ordered')
cosine_sim = utilities.plot_cosine_similarities(top_ims[clu_labs_rm_outliers_order][:top-list(clu_labs_rm_outliers).count(100)], min_sim=0, max_sim=1, maps = activations, label = 'ordered')
# for cluster in np.unique(clu_labs_rm_outliers):
#     if cluster == 100: 
#         pass
#     else:
#          cosine_sim = utilities.plot_cosine_similarities(top_ims[clu_labs_rm_outliers==cluster], min_sim=0, max_sim=1, maps = activations, label = f'cluster {round(cluster)}')

In [ ]:
distance_matrix = utilities.plot_euclidan_distances(top_activations, 0, 8, label = 'all')
# distance_matrix = utilities.plot_euclidan_distances(top_activations[clu_labs_rm_outliers_order], 0, 8, label = 'ordered')
distance_matrix = utilities.plot_euclidan_distances(top_activations[clu_labs_rm_outliers_order][:top-list(clu_labs_rm_outliers).count(100)], 0, 8, label = 'ordered')
# for cluster in np.unique(clu_labs_rm_outliers):
#     if cluster == 100: 
#         pass
#     else:
#         distance_matrix = utilities.plot_euclidan_distances(top_activations[clu_labs_rm_outliers==cluster], label = f'cluster {round(cluster)}')

plot images: 

https://stackoverflow.com/questions/22566284/matplotlib-how-to-plot-images-instead-of-points

https://matplotlib.org/stable/gallery/text_labels_and_annotations/demo_annotation_box.html

add jitter to reduce overlap:

https://stackoverflow.com/questions/8671808/matplotlib-avoiding-overlapping-datapoints-in-a-scatter-dot-beeswarm-plot

Visualise embeddings of images remaining in clusters

In [ ]:
XY_UMAP = umap.UMAP(n_components=2).fit_transform(top_activations[clu_labs_rm_outliers != 100])
amount = 0#.05
fig, ax = plt.subplots(figsize = (10,10))
#ax.set_title("UMAP")
ax.scatter(XY_UMAP[:,0], XY_UMAP[:,1]) 

for x0, y0, path in zip(utilities.rand_jitter(XY_UMAP[:,0], amount), utilities.rand_jitter(XY_UMAP[:,1], amount), [paths[i] for i in top_ims[clu_labs_rm_outliers != 100]]):
    ab = AnnotationBbox(utilities.getImage(path), (x0, y0), frameon=False)
    ax.add_artist(ab)
ax.axes.xaxis.set_ticklabels([])
ax.axes.yaxis.set_ticklabels([])
#ax.axis('off') 

Plot normalised concepts (removed as same plot below..)

In [ ]:
# for cluster in np.unique(clu_labs_rm_outliers):
#     if cluster == 100: 
#         pass
#     else:
#         plt.plot(top_activations[clu_labs_rm_outliers==cluster].mean(0)/np.linalg.norm(top_activations[clu_labs_rm_outliers==cluster].mean(0)))
#         plt.show()
#         print(f"Concept vector {round(cluster)}, direction {direction} activation: ", top_activations[clu_labs_rm_outliers==cluster].mean(0)[direction])
        

## Step 4: Calculate concept vectors

In [ ]:
concept_vecs = []
for cluster in np.unique(clu_labs_rm_outliers):
    if cluster == 100: 
        pass
    else:
        concept_vecs.append(top_activations[clu_labs_rm_outliers==cluster].mean(0)/np.linalg.norm(top_activations[clu_labs_rm_outliers==cluster].mean(0)))

In [ ]:
for i in range(len(concept_vecs)): 
    # print("cosine similarity: ", np.dot(concept_vecs[i], 
    concepts_dot = np.empty([len(concept_vecs),len(concept_vecs)])
    concepts_sim = np.empty([len(concept_vecs),len(concept_vecs)])
    for i in range(len(concept_vecs)):
        for j in range(len(concept_vecs)):
            concepts_dot[i,j] = np.dot(concept_vecs[i], concept_vecs[j])
            # same thing as concepts normalised length 1 
            concepts_sim[i,j] = concepts_dot[i,j]/(np.linalg.norm(concept_vecs[i])*np.linalg.norm(concept_vecs[j])) 
# print(concepts_dot)

### Analysis of concept vectors

In [ ]:
for idx, cluster in enumerate(np.unique(clu_labs_rm_outliers)):
    if cluster == 100: 
        pass
    else:
        plt.plot(concept_vecs[idx])
        plt.show()
        print(f"Concept vector {round(cluster)}, direction {direction} activation: ", top_activations[clu_labs_rm_outliers==cluster].mean(0)[direction])

In [ ]:
print(concepts_sim)
ax = plt.subplot()
im = ax.imshow(concepts_sim, cmap='viridis', interpolation='nearest', vmin=0, vmax=1) 
plt.title(f"Concept vector cosine similarities")
plt.subplots_adjust(right=0.8)
cbar_ax = plt.axes([0.85, 0.1, 0.075, 0.8])
plt.colorbar(mappable=(im), cax=cbar_ax)
plt.show()

Projection of top activating direction images on concept vectors vs vanilla direction

In [ ]:
for cluster in np.unique(clu_labs_rm_outliers):
    if cluster == 100: 
        pass
    else:
        concept_vec = top_activations[clu_labs_rm_outliers==cluster].mean(0)/np.linalg.norm(top_activations[clu_labs_rm_outliers==cluster].mean(0))
        cluster_vecs = top_activations[clu_labs_rm_outliers==cluster]
        concept_dot = [np.dot(concept_vec, cluster_vecs[i]) for i in range(cluster_vecs.shape[0])]
        #print("\nconcept_dot - projections along concept: ",concept_dot)
        concept_sim = [concept_dot[i]/np.linalg.norm(cluster_vecs[i]) for i in range(cluster_vecs.shape[0])]
        #print("\nconcept_sim - cosine similarity with concept: ",concept_sim)

        direction_dot = cluster_vecs[:,direction]
        #print("\ndirection_dot - projections along direction: ", direction_dot)
        direction_sim = [direction_dot[i]/np.linalg.norm(cluster_vecs[i]) for i in range(cluster_vecs.shape[0])]
        #print("\ndirection_sim - cosine similarity with direction: ",direction_sim)
        fig, axs = plt.subplots(1, 2, figsize=(2*2, 2 * 1))
        axs[0].set_title(f"projection", fontsize=10)
        axs[0].tick_params(axis='both', which='major', labelsize=8)
        axs[0].tick_params(axis='both', which='minor', labelsize=8)
        sns.boxplot((concept_dot, direction_dot),ax = axs[0])
        axs[1].set_title(f"cosine similarity", fontsize=10)
        axs[1].tick_params(axis='both', which='major', labelsize=8)
        axs[1].tick_params(axis='both', which='minor', labelsize=8)
        sns.boxplot((concept_sim, direction_sim),ax = axs[1])
        plt.tight_layout()


### Maximally projecting images along concept directions

Find images with largest projection along concept direction. This is effecttively finding the maximally activating images for these directions. 

Check these directions are clean, monosemantic regions. 

In [ ]:
concepts_ims_dot = np.empty([len(conv_maps_avg),len(concept_vecs)])
concepts_ims_sim = np.empty([len(conv_maps_avg),len(concept_vecs)])
for i in range(len(conv_maps_avg)):
    for concept_id in range(len(concept_vecs)):
        concepts_ims_dot[i,concept_id] = np.dot(conv_maps_avg[i], concept_vecs[concept_id])
        concepts_ims_sim[i,concept_id] = concepts_ims_dot[i,concept_id]/(np.linalg.norm(conv_maps_avg[i])*np.linalg.norm(conv_maps_avg[concept_id]))


In [ ]:
top_projs = []
for concept in range(len(concepts_ims_dot[0,])):
    top_projs.append(concepts_ims_dot[:,concept].argsort()[-top:][::-1])

In [ ]:
# top_projs

In [ ]:
for concept in range(len(concept_vecs)):
    fig, ax = plt.subplots(math.ceil(top//5), 5, figsize = (10,20))
    ax = ax.flatten()
    for idx, im_id in enumerate(top_projs[concept]):# enumerate(concepts_dot[:,concept].argsort()[-top:][::-1]):
        im = Image.open(paths[im_id])
        ax[idx].imshow(im)
        ax[idx].set_title(f"{im_id}", size = 8)
        ax[idx].axis('off')

In [ ]:
#Compared to the original top activating images for the direction:
# print(top_ims)
# top_ims = utilities.get_activations(activations_avg = activations, direction = direction).argsort()[-top:][::-1] 
# for i in range(50):
#         im = Image.open(paths[top_ims[i]])
#         plt.imshow(im)
#         plt.axis('off')
#         plt.show()

## Introducing concept sparsity

In [ ]:
threshold = 0.1

In [ ]:
concept_vecs_th = copy.deepcopy(concept_vecs)
for concept in range(len(concept_vecs_th)):
    l_th = concept_vecs_th[concept]<threshold
    concept_vecs_th[concept][l_th] = 0
    concept_vecs_th[concept] = concept_vecs_th[concept]/np.linalg.norm(concept_vecs_th[concept])

#plt.plot(concept_vecs[0][concept_vecs[0]>0.1])

In [ ]:
for i in range(len(concept_vecs)): 
    # print("cosine similarity: ", np.dot(concept_vecs[i], 
    concepts_dot_th = np.empty([len(concept_vecs_th),len(concept_vecs_th)])
    concepts_sim_th = np.empty([len(concept_vecs_th),len(concept_vecs_th)])
    for i in range(len(concept_vecs_th)):
        for j in range(len(concept_vecs_th)):
            concepts_dot_th[i,j] = np.dot(concept_vecs_th[i], concept_vecs_th[j])
            # same thing as concepts normalised length 1 
            concepts_sim_th[i,j] = concepts_dot_th[i,j]/(np.linalg.norm(concept_vecs_th[i])*np.linalg.norm(concept_vecs_th[j])) 
# print(concepts_dot)

In [ ]:
for idx, cluster in enumerate(np.unique(clu_labs_rm_outliers)):
    if cluster == 100: 
        pass
    else:
        plt.plot(concept_vecs_th[idx])
        plt.show()
        print(f"Concept vector {round(cluster)}, direction {direction} activation: ", top_activations[clu_labs_rm_outliers==cluster].mean(0)[direction])

In [ ]:
print(concepts_sim_th)
ax = plt.subplot()
im = ax.imshow(concepts_sim_th, cmap='viridis', interpolation='nearest', vmin=0, vmax=1) 
plt.title(f"Concept vector cosine similarities")
plt.subplots_adjust(right=0.8)
cbar_ax = plt.axes([0.85, 0.1, 0.075, 0.8])
plt.colorbar(mappable=(im), cax=cbar_ax)
plt.show()

In [ ]:
concepts_th_dot = np.empty([len(conv_maps_avg),len(concept_vecs_th)])
concepts_th_sim = np.empty([len(conv_maps_avg),len(concept_vecs_th)])
for i in range(len(conv_maps_avg)):
    for concept_id in range(len(concept_vecs_th)):
        concepts_th_dot[i,concept_id] = np.dot(conv_maps_avg[i], concept_vecs_th[concept_id])
        concepts_th_sim[i,concept_id] = concepts_th_dot[i,concept_id]/(np.linalg.norm(conv_maps_avg[i])*np.linalg.norm(conv_maps_avg[concept_id]))
    

In [ ]:
top_projs_th = []
for concept in range(len(concepts_th_dot[0,])):
    top_projs_th.append(concepts_th_dot[:,concept].argsort()[-top:][::-1])

In [ ]:
for concept in range(len(concept_vecs_th)):
    fig, ax = plt.subplots(math.ceil(top//5), 5, figsize = (10,20))
    ax = ax.flatten() 
    for idx, im_id in enumerate(top_projs_th[concept]):# enumerate(concepts_dot[:,concept].argsort()[-top:][::-1]):
        im = Image.open(paths[im_id])
        ax[idx].imshow(im)
        ax[idx].set_title(f"{im_id}", size = 8)
        ax[idx].axis('off')

In [ ]:
# for i in range(len(concept_vecs)): 
#     # print("cosine similarity: ", np.dot(concept_vecs[i], 
#     concepts_dot = np.empty([len(concept_vecs),len(concept_vecs)])
#     concepts_sim = np.empty([len(concept_vecs),len(concept_vecs)])
#     for i in range(len(concept_vecs)):
#         for j in range(len(concept_vecs)):
#             concepts_dot[i,j] = np.dot(concept_vecs[i], concept_vecs[j])
#             # same thing as concepts normalised length 1 
#             concepts_sim[i,j] = concepts_dot[i,j]/(np.linalg.norm(concept_vecs[i])*np.linalg.norm(concept_vecs[j])) 
# # print(concepts_dot)

In [ ]:
for cluster in np.unique(clu_labs_rm_outliers):
    i=0
    if cluster == 100: 
        pass
    else:
        concept_vec = concept_vecs_th[i]
        cluster_vecs = top_activations[clu_labs_rm_outliers==cluster]
        concept_dot = [np.dot(concept_vec, cluster_vecs[i]) for i in range(cluster_vecs.shape[0])]
        #print("\nconcept_dot - projections along concept: ",concept_dot)
        concept_sim = [concept_dot[i]/np.linalg.norm(cluster_vecs[i]) for i in range(cluster_vecs.shape[0])]
        #print("\nconcept_sim - cosine similarity with concept: ",concept_sim)

        direction_dot = cluster_vecs[:,direction]
        #print("\ndirection_dot - projections along direction: ", direction_dot)
        direction_sim = [direction_dot[i]/np.linalg.norm(cluster_vecs[i]) for i in range(cluster_vecs.shape[0])]
        #print("\ndirection_sim - cosine similarity with direction: ",direction_sim)
        fig, axs = plt.subplots(1, 2, figsize=(2*2, 2 * 1))
        axs[0].set_title(f"projection", fontsize=10)
        axs[0].tick_params(axis='both', which='major', labelsize=8)
        axs[0].tick_params(axis='both', which='minor', labelsize=8)
        sns.boxplot((concept_dot, direction_dot),ax = axs[0])
        axs[1].set_title(f"cosine similarity", fontsize=10)
        axs[1].tick_params(axis='both', which='major', labelsize=8)
        axs[1].tick_params(axis='both', which='minor', labelsize=8)
        sns.boxplot((concept_sim, direction_sim),ax = axs[1])
        plt.tight_layout()
    i+=1


## Human in the loop

Generate 5 images for each cluster for qualitative assessment of concepts. 

In [ ]:
# count observations in each cluster as we randomly select 5 of these images with a random seed
# count is a list of the number of observations to choose from in each cluster
random.seed(0)
count = []
for cluster in np.unique(clu_labs_rm_outliers):
    if cluster == 100: 
        pass
    else:
        count_cl = 0
        for i in range(len(clu_labs_rm_outliers)):
            if clu_labs_rm_outliers[i] == cluster:
                count_cl += 1       
        count.append(count_cl) 
        
# print 5 images of each concept, save manually and use in study
for idx, cluster in enumerate(np.unique(clu_labs_rm_outliers)):
    if cluster == 100: 
        pass
    else:
        print(f"Cluster {round(cluster)} samples: ")
        for samples_ in range(5):
            if idx == 0: 
                rand_cl = random.randint(0,count[idx]) 
            else:
                rand_cl = random.randint(sum(count[:idx]), sum(count[:idx+1]))
            print(rand_cl)
            im_id = top_ims[clu_labs_rm_outliers_order][rand_cl-1]
            im = Image.open(paths[im_id])
            im.show()

Generate 5 top projecting images for each concept for qualitative assessment of concepts. 

In [ ]:
for concept in range(len(top_projs)):
    print(f"Concept {round(concept)} samples: ")
    for samples_ in range(5):
        rand_im = random.randint(0,top) 
        print(rand_im)
        im_id = top_projs[concept][rand_im]
        im = Image.open(paths[im_id])
        im.show()
#top_projs[concept]

In [ ]:
# print 5 images from each cluster, save manually and use in study
for idx, cluster in enumerate(np.unique(clu_labs_rm_outliers)):
    if cluster == 100: 
        pass
    else:
        print(f"Cluster {round(cluster)} samples: ")
        for samples_ in range(5):
            if idx == 0: 
                rand_cl = random.randint(0,count[idx]) 
            else:
                rand_cl = random.randint(sum(count[:idx]), sum(count[:idx+1]))
            print(rand_cl)
            im_id = top_ims[clu_labs_rm_outliers_order][rand_cl-1]
            im = Image.open(paths[im_id])
            im.show()

## Feature visualisation

In [ ]:
# !pip uninstall scripts-inceptionv3
#!pip install -e .

In [ ]:
torch.hub.load('pytorch/vision:v0.9.0', 'inception_v3', pretrained=True).state_dict()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.hub.load('pytorch/vision:v0.9.0', 'inception_v3', pretrained=True)
model.to(device).eval()
# tried... 
# import inceptionv3
# imp.reload(inceptionv3) 
# print("model")
# inceptionv3.InceptionV3.Inception5h(pretrained=True)
# torch.hub.load('pytorch/vision:v0.9.0', 'inception_v3', pretrained=True, progress=True).state_dict()
# model = inceptionv3.InceptionV3.Inception5h(pretrained=True)

In [ ]:
from lucent.modelzoo.util import get_model_layers

In [ ]:
get_model_layers(model)

In [ ]:
layer='Mixed_7b'

direction objective

In [ ]:
batch_param_f = lambda: param.image(128, batch=6, decorrelate=True)
obj = objectives.channel(layer, direction)
_ = render.render_vis(model, obj, batch_param_f, show_inline=True, verbose=True)

In [ ]:
batch_param_f = lambda: param.image(128, batch=6, decorrelate=True)
obj = objectives.channel(layer, direction) - 0.2 * objectives.diversity(layer)
_ = render.render_vis(model, obj, batch_param_f, show_inline=True, verbose=True)

cluster direction objectives

In [ ]:
concept_vecs

In [ ]:
i = 0
for concept in range(len(concept_vecs)):
    batch_param_f = lambda: param.image(128, batch=6, decorrelate=True)
    obj = objectives.direction(layer, torch.tensor(concept_vecs[i]).to(device)) # round(cluster) 
    _ = render.render_vis(model, obj, batch_param_f, show_inline=True, verbose=True)
    i+=1

In [ ]:
i = 0
for concept in range(len(concept_vecs)):
    batch_param_f = lambda: param.image(128, batch=6, decorrelate=True)
    obj = objectives.direction(layer, torch.tensor(concept_vecs[i]).to(device)) # round(cluster) 
    _ = render.render_vis(model, obj, batch_param_f, transforms=[transform.jitter(2)], show_inline=True, verbose=True)
    i+=1

In [ ]:
i = 0
for concept in range(len(concept_vecs)):
    batch_param_f = lambda: param.image(128, batch=6, decorrelate=True)
    obj = objectives.direction(layer, torch.tensor(concept_vecs[i]).to(device)) - 0.2 * objectives.diversity(layer)
    _ = render.render_vis(model, obj, batch_param_f, transforms=[transform.jitter(2)], show_inline=True, verbose=True)
    i+=1

### Random analysis

In [ ]:
#test
# clu_labs
# print(clu_lab_order)
# clu_labs_rm_outliers
# clu_labs_rm_outliers_order = sorted(range(len(clu_labs_rm_outliers)), key=lambda k: clu_labs_rm_outliers[k])
# print(clu_labs_rm_outliers[clu_labs_rm_outliers_order])
# print(clu_labs_rm_outliers_order)
# clu_labs_rm_outliers
# clu_labs_rm_outliers
# clu_labs_rm_outliers[clu_labs_rm_outliers != 100]

In [ ]:
print("clusterer.n_clusters_: ", clusterer.n_clusters_)
print("clusterer.n_leaves_: ", clusterer.n_leaves_)
print("clusterer.children_: ", clusterer.children_)
print("clusterer.distances_: ", clusterer.distances_)

 ### RW hierarchial and kmeans clustering

In [ ]:
# print("clusterer_0.distances_", clusterer_0.distances_)
# print("y", y)
# format for displaying
# y_format = ['{:3f}'.format(clusterer_0.distances_[i] - clusterer_0.distances_[i-1]) if i else '{:3f}'.format(clusterer_0.distances_[i]) for i in range(len(clusterer_0.distances_))]  
# print("y_format", y_format)
# max(y)
# np.argmax(y) # 98
# clusterer.children_[np.argmax(y)]
# clusterer_0.distances_[np.argmax(y)]
# clusterer_0.distances_[np.argmax(y)] - clusterer_0.distances_[np.argmax(y)-1]

In [ ]:
#Rough
# linkage(np.asarray(top_activations), method='ward', metric='euclidean')
# linkage_res # (99, 4)
# scipy.cluster.hierarchy.cophenet()

## Save images

In [ ]:
# create and empty folders
if not os.path.exists(f'{SAVEFOLD}concept_ims_direction_{direction}/'):
    os.mkdir(f'{SAVEFOLD}concept_ims_direction_{direction}/')
if not os.path.exists(f'{SAVEFOLD}concept_ims_direction_{direction}/top_ims/'):
    os.mkdir(f'{SAVEFOLD}concept_ims_direction_{direction}/top_ims')
        
folder = f'{SAVEFOLD}concept_ims_direction_{direction}/top_ims/'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
# create and empty folders for top images
if not os.path.exists(f'{SAVEFOLD}concept_ims_direction_{direction}/'):
    os.mkdir(f'{SAVEFOLD}concept_ims_direction_{direction}/')
if not os.path.exists(f'{SAVEFOLD}concept_ims_direction_{direction}/top_ims/'):
    os.mkdir(f'{SAVEFOLD}concept_ims_direction_{direction}/top_ims')
        
folder = f'{SAVEFOLD}concept_ims_direction_{direction}/top_ims/'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))
        
# create and empty folders for concepts and ransom
concepts = [f'cluster_{round(i)}' for i in np.unique(clu_labs_rm_outliers)] + [f'random_{round(i)}' for i in np.unique(clu_labs_rm_outliers)]
for concept in concepts:
    if not os.path.exists(f'{SAVEFOLD}concept_ims_direction_{direction}/{concept}/'):
        os.mkdir(f'{SAVEFOLD}concept_ims_direction_{direction}/{concept}/')
    folder = f'{SAVEFOLD}concept_ims_direction_{direction}/{concept}/'
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
# populate folders
for cluster in np.unique(clu_labs_rm_outliers):
    if cluster == 100: 
        pass
    else:
        top_ims[clu_labs_rm_outliers==cluster]

for idx, im_id in enumerate(top_ims[clu_lab_order]):
    im = Image.open(paths[im_id])
    im.save(f"{SAVEFOLD}concept_ims_direction_{direction}/top_ims/{os.path.basename(os.path.normpath(paths[im_id]))}")

for cluster in np.unique(clu_labs_rm_outliers):
    if cluster == 100: 
        pass
    else:
        for idx, im_id in enumerate(top_ims[clu_labs_rm_outliers==cluster]): 
            im = Image.open(paths[im_id])
            im.save(f"{SAVEFOLD}concept_ims_direction_{direction}/cluster_{round(cluster)}/{os.path.basename(os.path.normpath(paths[im_id]))}")
    
# old   
# for idx, im_id in enumerate(top_ims[clu_lab_order]):
#     if clu_labs[clu_lab_order][idx] == 0:
#         im = Image.open(paths[im_id])
#         im.save(f"{SAVEFOLD}concept_ims_direction_{direction}/cluster_0/{os.path.basename(os.path.normpath(paths[im_id]))}")
#     elif clu_labs[clu_lab_order][idx] == 1:
#         im = Image.open(paths[im_id])
#         im.save(f"{SAVEFOLD}concept_ims_direction_{direction}/cluster_1/{os.path.basename(os.path.normpath(paths[im_id]))}")

In [ ]:
# populate folders of random images to test concepts against
for cluster in np.unique(clu_labs_rm_outliers):
    if cluster == 100: 
        pass
    else:
        for idx, im_id in enumerate(top_ims[clu_labs_rm_outliers==cluster]): 
            rand = random.randint(0, len(paths))
            im = Image.open(paths[rand])
            im.save(f"{SAVEFOLD}concept_ims_direction_{direction}/random_{round(cluster)}/{os.path.basename(os.path.normpath(paths[rand]))}")


In [ ]:
end runnable script

In [ ]:
kmeans.cluster_centers_

## Other

### kmeans

How many clusters for kmeans?

Plot the sum of squared distances from the data points to the centers of the k-Means clusters for various values of k. Use the Elbow method to pick the best value of k. 

In [ ]:
kmax = 10

In [ ]:
sse = {}
for k in range(1,kmax):
        kmeans = cluster.KMeans(n_clusters=k, n_init=10, max_iter=1000, 
                                random_state=1).fit(top_activations)
        sse[k] = kmeans.inertia_ 
        # Inertia: Sum of distances of samples to their closest cluster center
        # label = kmeans.labels_
        # sil_coeff[k] = silhouette_score(data, label, metric='euclidean')

In [ ]:
sse

In [ ]:
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()), 'o-')
plt.xlabel("Number of clusters")
plt.ylabel("SSE")
plt.title("Elbow Method")
plt.show()

In [ ]:
sil_coeff = {}
for k in range(2,kmax):
    random.seed(1)
    kmeans = cluster.KMeans(n_clusters=k, n_init=10, max_iter=1000,
                            random_state=2021).fit(top_activations)
    label = kmeans.labels_
    sil_coeff[k] = silhouette_score(top_activations, label, metric='euclidean')

In [ ]:
sil_coeff

In [ ]:
plt.figure()
plt.plot(list(sil_coeff.keys()), list(sil_coeff.values()), "o-")
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.title("Silhouette width")
plt.show()

## Dimensionality reduction

In [ ]:
XY_MDS = manifold.MDS(n_components=2).fit_transform(top_activations)
plt.scatter(x=XY_MDS[:,0],y=XY_MDS[:,1])

In [ ]:
XY_TSNE = manifold.TSNE(n_components=2,perplexity=10).fit_transform(top_activations)
plt.scatter(x=XY_TSNE[:,0],y=XY_TSNE[:,1])

In [ ]:
XY_UMAP = umap.UMAP(n_components=2).fit_transform(top_activations)
plt.scatter(x=XY_UMAP[:,0],y=XY_UMAP[:,1])

## RW

### Trying HDBSCAN 
unsuccessful so far... CLusters either in one cluster, or all outlers usually if allow_single_cluster = True. If you don't allow a single cluster, although it gives multiple clusters, it doesn't cluster the concepts very well. 

In [ ]:
# HDBSCAN doesn't work for top_activations, UMAP 2D, UMAP 10 D
XY_UMAP = umap.UMAP(n_components=3).fit_transform(top_activations) # CHANGE
clusterer = hdbscan.HDBSCAN() # allow_single_cluster = True
clusterer.fit(XY_UMAP) # top_activations
print(clusterer.labels_)
utilities.clustering_scatterplot(points=XY_UMAP, 
                       labels=clusterer.labels_,
                       centers=None, 
                       title='UMAP')

In [ ]:
clu_labs = clusterer.labels_
print(clu_labs)
clu_lab_order = sorted(range(len(clu_labs)), key=lambda k: clu_labs[k])

fig, ax = plt.subplots(math.ceil(len(top_ims)//5), 5, figsize = (10,20))
ax = ax.flatten()
for idx, im_id in enumerate(top_ims[clu_lab_order]):
    im = Image.open(paths[im_id])
    ax[idx].imshow(im)
    ax[idx].set_title(f"{im_id}: cluster {clu_labs[clu_lab_order][idx]}", size = 8)
    ax[idx].axis('off')

In [ ]:
# # 
# distance_matrix = pairwise_distances(top_activations, metric = 'euclidean')
# clusterer = hdbscan.HDBSCAN(metric='precomputed', allow_single_cluster = True)
# clusterer.fit(distance_matrix)
# print(clusterer.labels_)
# print(distance_matrix)
# print(np.min(distance_matrix[np.nonzero(distance_matrix)]))
# utilities.clustering_scatterplot(points=XY_UMAP, 
#                        labels=clusterer.labels_,
#                        centers=None, 
#                        title='UMAP')

### Trying DBSCAN
unsuccessful so far... 

In [ ]:
# clustered_data_sklearn = DBSCAN(eps=10).fit(top_activations) # , metric = "cosine"
# print(clustered_data_sklearn.labels_)
# clustered_data_sklearn

# DBSCAN doesn't work for top_activations, UMAP 2D, UMAP 10D, cosine UMAP 2D, cosine UMAP 10D
clusterer = DBSCAN(metric = "cosine") 
clusterer.fit(XY_UMAP) # top_activations
print(clusterer.labels_)
utilities.clustering_scatterplot(points=XY_UMAP, 
                       labels=clusterer.labels_,
                       centers=None, 
                       title='UMAP')

In [ ]:
clu_labs = clusterer.labels_
print(clu_labs)
clu_lab_order = sorted(range(len(clu_labs)), key=lambda k: clu_labs[k])

fig, ax = plt.subplots(math.ceil(len(top_ims)//5), 5, figsize = (10,20))
ax = ax.flatten()
for idx, im_id in enumerate(top_ims[clu_lab_order]):
    im = Image.open(paths[im_id])
    ax[idx].imshow(im)
    ax[idx].set_title(f"{im_id}: cluster {clu_labs[clu_lab_order][idx]}", size = 8)
    ax[idx].axis('off')

### Analyse n_clusters clusters

In [ ]:
n_clusters = 2

Increased the max_iter to 1000 to allow the algorithm more time to converge. 
Increased n_init to run the algorithm more times with various random seeds. 
The final results is then the best output of 100 consecutive runs in terms of inertia.

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
# Append the cluster centers to the dataset.
clustered_data_sklearn = KMeans(n_clusters=n_clusters, n_init=100, max_iter=1000).fit(top_activations)

In [ ]:
data_and_centers = np.r_[top_activations,clustered_data_sklearn.cluster_centers_]

In [ ]:
top_activations

Apply a manifold-learning technique to project the data set to a 2D space. 

In [ ]:
# Apply multi-dimensional scaling (MDS) to project both the data and the k-Means cluster centers to a 2D space
XYcoordinates = manifold.MDS(n_components=2).fit_transform(data_and_centers)
print("transformation complete")

In [ ]:
utilities.clustering_scatterplot(points=XYcoordinates[:-n_clusters,:], 
                       labels=clustered_data_sklearn.labels_, 
                       centers=XYcoordinates[-n_clusters:,:], 
                       title='MDS')

In [ ]:
# Apply t-SNE to project both the data and the k-Means cluster centers to a 2D space


In [ ]:
XYcoordinates = manifold.TSNE(n_components=2, perplexity=10).fit_transform(data_and_centers)
print("transformation complete")

In [ ]:
clustering_scatterplot(points=XYcoordinates[:-n_clusters,:], 
                       labels=clustered_data_sklearn.labels_,
                       centers=XYcoordinates[-n_clusters:,:], 
                       title='TSNE')

It is unclear exactly where the elbow is here. I will therefore look at the average silhouette width for different numbers of clusters. This gives a measure of how well defined clusters are (points in clusters being more similar and points in different clusters being more different). 

In [ ]:
clu_labs = clustered_data_sklearn.labels_
# clu_labs

In [ ]:
# fig, ax = plt.subplots(math.ceil(len(top_ims)//5), 5)
# ax = ax.flatten()
# for idx, im_id in enumerate(top_ims):
#     im = Image.open(paths[im_id])
#     ax[idx].imshow(im)
#     ax[idx].set_title(f"{im_id}: cluster {clu_labs[idx]}", size = 8)
#     ax[idx].axis('off')

In [ ]:
clu_lab_order = sorted(range(len(clustered_data_sklearn.labels_)), key=lambda k: clustered_data_sklearn.labels_[k])
# clu_lab_order

In [ ]:
fig, ax = plt.subplots(math.ceil(len(top_ims)//5), 5, figsize = (10,20))
ax = ax.flatten()
for idx, im_id in enumerate(top_ims[clu_lab_order]):
    im = Image.open(paths[im_id])
    ax[idx].imshow(im)
    ax[idx].set_title(f"{im_id}: cluster {clu_labs[clu_lab_order][idx]}", size = 8)
    ax[idx].axis('off')

In [ ]:
# clu_lab_order

# clu_labs[clu_lab_order] # array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#        1, 1, 1], dtype=int32)

# for idx, im_id in enumerate(top_ims[clu_lab_order]):
#     print(im_id)